## Demo of fmm Python API

In [1]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT

### Load network data and graph

In [2]:
import os
PWD = os.getcwd() # os.path.abspath(os.path.dirname(__file__))
network = Network(f"{PWD}/../data/edges.shp")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

Nodes 17 edges 30
[2023-06-04 11:14:25.747] [info] [network.cpp:155] Read network from file /home/tzx/git/fmm/example/notebook/../data/edges.shp
[2023-06-04 11:14:25.774] [info] [network.cpp:232] Number of edges 30 nodes 17
[2023-06-04 11:14:25.774] [info] [network.cpp:233] Field index: id 1 source 2 target 3
[2023-06-04 11:14:25.774] [info] [network.cpp:236] Read network done.
[2023-06-04 11:14:25.776] [info] [network_graph.cpp:18] Construct graph from network edges start
[2023-06-04 11:14:25.776] [info] [network_graph.cpp:31] Graph nodes 17 edges 30
[2023-06-04 11:14:25.776] [info] [network_graph.cpp:32] Construct graph from network edges end


In [3]:
dumps = network.dumps()
print(dumps[:500])
print(len(dumps))

{"srid":4326,"edges":[{"id":1,"source":1,"target":2,"coordinates":[[2.0,1.0],[2.0,0.0]]},{"id":2,"source":2,"target":1,"coordinates":[[2.0,0.0],[2.0,1.0]]},{"id":3,"source":3,"target":1,"coordinates":[[3.0,1.0],[2.0,1.0]]},{"id":4,"source":4,"target":3,"coordinates":[[4.0,1.0],[3.0,1.0]]},{"id":5,"source":1,"target":5,"coordinates":[[2.0,1.0],[2.0,2.0]]},{"id":6,"source":5,"target":1,"coordinates":[[2.0,2.0],[2.0,1.0]]},{"id":7,"source":3,"target":6,"coordinates":[[3.0,1.0],[3.0,2.0]]},{"id":8,"
2097


In [4]:
# from/to_json
nn = Network()
nn.loads(dumps)
redumps = nn.dumps()
assert redumps == dumps, f'old: {dumps},\nnew: {redumps}'
# network.dump('network.json')

[2023-06-04 11:14:25.797] [info] [network.cpp:113] Number of edges 30 nodes 17
[2023-06-04 11:14:25.797] [info] [network.cpp:115] Read network done.


### Precompute an UBODT file

**This step can be skipped if you already created one UBODT file.**

In [5]:
from fmm import UBODTGenAlgorithm

In [6]:
ubodt_gen = UBODTGenAlgorithm(network,graph)

In [7]:
status = ubodt_gen.generate_ubodt(f"{PWD}/../data/ubodt.txt", 4, binary=False, use_omp=True)
print(status)

[2023-06-04 11:14:25.825] [info] [ubodt_gen_algorithm.cpp:94] Start to generate UBODT with delta 4
[2023-06-04 11:14:25.825] [info] [ubodt_gen_algorithm.cpp:95] Output format csv
[2023-06-04 11:14:25.837] [info] [ubodt_gen_algorithm.cpp:124] Progress 10 / 17
Status: success
Time takes 0.029 seconds



In [24]:
path = f"{PWD}/../data/ubodt.json"
status = ubodt_gen.generate_ubodt(path, 4, binary=False)
print(status)
print(path)

Status: success
Time takes 0 seconds
[2023-06-04 11:14:43.391] [info] [ubodt_gen_algorithm.cpp:156] Start to generate UBODT with delta 4
[2023-06-04 11:14:43.391] [info] [ubodt_gen_algorithm.cpp:157] Output format json
[2023-06-04 11:14:43.391] [info] [ubodt_gen_algorithm.cpp:160] Progress 0 / 17

/home/tzx/git/fmm/example/notebook/../data/ubodt.json
[2023-06-04 11:14:43.391] [info] [ubodt_gen_algorithm.cpp:160] Progress 10 / 17


### Load UBODT data

In [9]:
ubodt = UBODT.read_ubodt_csv(f"{PWD}/../data/ubodt.txt")

[2023-06-04 11:14:25.883] [info] [ubodt.cpp:225] Reading UBODT file (CSV format) from /home/tzx/git/fmm/example/notebook/../data/ubodt.txt
[2023-06-04 11:14:25.883] [info] [ubodt.cpp:255] Finish reading UBODT with rows 138


### Create FMM model

In [10]:
model = FastMapMatch(network,graph,ubodt)

### Define FMM configuration

In [11]:
k = 4
radius = 0.4
gps_error = 0.5
fmm_config = FastMapMatchConfig(k,radius,gps_error)

### Run map matching

#### Match a single trajectory

In [12]:
wkt = "LINESTRING(0.200812146892656 2.14088983050848,1.44262005649717 2.14879943502825,3.06408898305084 2.16066384180791,3.06408898305084 2.7103813559322,3.70872175141242 2.97930790960452,4.11606638418078 2.62337570621469)"

In [13]:
result = model.match_wkt(wkt,fmm_config)

In [14]:
print("Matched path: ", list(result.cpath))
print("Matched edge for each point: ", list(result.opath))
print("Matched edge index ",list(result.indices))
print("Matched geometry: ",result.mgeom.export_wkt())
print("Matched point ", result.pgeom.export_wkt())

Matched path:  [8, 11, 13, 18, 20, 24]
Matched edge for each point:  [8, 11, 18, 18, 20, 24]
Matched edge index  [0, 1, 3, 3, 4, 5]
Matched geometry:  LINESTRING(0.20081215 2,1 2,2 2,3 2,3 3,4 3,4 2.6233757)
Matched point  LINESTRING(0.20081215 2,1.4426201 2,3 2.1606638,3 2.7103814,3.7087218 3,4 2.6233757)


##### Print detailed match information

In [15]:
candidates = []
for c in result.candidates:
    candidates.append((c.edge_id,c.source,c.target,c.error,c.length,c.offset,c.spdist,c.ep,c.tp))

In [16]:
import pandas as pd
df = pd.DataFrame(candidates,
                  columns=["eid","source","target","error","length","offset","spdist","ep","tp"])

In [17]:
df.head()

,eid,source,target,error,length,offset,spdist,ep,tp
0,8,7,8,0.140890,1.0,0.200812,0.000000,0.961078,0.000000
1,11,8,5,0.148799,1.0,0.442620,1.241808,0.956684,1.000000
2,18,6,11,0.064089,1.0,0.160664,1.718044,0.991819,0.943813
3,18,6,11,0.064089,1.0,0.710381,0.549718,0.991819,1.000000
4,20,11,12,0.020692,1.0,0.708722,0.998340,0.999144,0.699640


#### Match trajectories in a GPS file

In [18]:
from fmm import GPSConfig,ResultConfig

In [19]:
input_config = GPSConfig()
input_config.file = f"{PWD}/../data/trips.csv"
input_config.id = "id"

In [20]:
print(input_config.to_string())

gps file : /home/tzx/git/fmm/example/notebook/../data/trips.csv
id column : id
geom column : geom
timestamp column : timestamp
x column : x
y column : y
GPS point : false



In [21]:
result_config = ResultConfig()
result_config.file = f"{PWD}/../data/mr.txt"
result_config.output_config.write_opath = True
print(result_config.to_string())

Result file : /home/tzx/git/fmm/example/notebook/../data/mr.txt
Output fields: opath cpath mgeom 


In [22]:
status = model.match_gps_file(input_config, result_config, fmm_config)

[2023-06-04 11:14:26.322] [warning] [result_config.cpp:226] Overwrite existing result file /home/tzx/git/fmm/example/notebook/../data/mr.txt
[2023-06-04 11:14:26.323] [info] [gps_reader.cpp:344] GPS data in trajectory CSV format
[2023-06-04 11:14:26.323] [warning] [gps_reader.cpp:147] Timestamp column timestamp not found
[2023-06-04 11:14:26.323] [info] [gps_reader.cpp:149] Id index 0 Geometry index 1 Timstamp index -1


In [23]:
print(status)

Status: success
Time takes 0.004 seconds
Total points 17 matched 17
Total trajectories 3 matched 3
Map match percentage 1
Map match speed 4250 points/s 

